# 라이브러리

In [18]:
# 파이썬 표준 라이브러리
import os
import requests
import re
import random
import time
from datetime import datetime
from pprint import pprint
from concurrent import futures

# 파이썬 서드파티 라이브러리
import bs4
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

# 전역 변수

In [7]:
# cpu 갯수
workers = os.cpu_count()
print(workers)

8


In [8]:
news_websites_dict = {
                    'investing': 'https://kr.investing.com/news/cryptocurrency-news',
                    'hankyung': 'https://www.hankyung.com/koreamarket/news/crypto',
                    'bloomingbit': 'https://bloomingbit.io/feed',
                    'coinreaders': 'https://www.coinreaders.com/',
                    'cryptonews': 'https://cryptonews.com/kr/news/',
                      }

In [9]:
# Chrome 옵션 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# User-Agent 변경을 위한 옵션 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
headers = {'User-Agent': user_agent}
options.add_argument(f"user-agent={user_agent}")

# 함수

In [ ]:
def hankyung(news_url: bs4.element.Tag) -> dict[str, str, None]:
    """뉴스 URL을 바탕으로 크롤링을 하는 함수

    Args:
        news_url: 뉴스 URL

    Returns:
        {
            "news_title": 뉴스 제목, str
            "news_first_upload_time": 뉴스 최초 업로드 시각, str | None
            "newsfinal_upload_time": 뉴스 최종 수정 시각, str
            "author": 뉴스 작성자
            "news_content": 뉴스 본문, str
            "news_url": 뉴스 URL, str
            "news_website": 뉴스 웹사이트, str
        }
    """
    info = {} # 뉴스 데이터 정보 Dictionary

    # news_url를 찾아서 requests로 HTML GET을 한 다음, BeautifulSoup로 parser
    url = news_url.find("a")["href"]
    html = requests.get(url).text
    time.sleep(0.1)
    soup = BeautifulSoup(html, 'html.parser')

    # 1. 뉴스 데이터의 제목
    title = soup.find('h1', class_='headline').text

    # 2. 뉴스 데이터의 최초 업로드 시각과 최종 수정 시각
    first_upload_time = None
    div = soup.find_all('span', class_='txt-date')
    span = div[1].find('span')
    last_upload_time_list = re.split(r'\s+\r\n\s+', span.text)[1].split()
    y_m_d = '-'.join(times[:-1] for times in last_upload_time_list[:3])
    if last_upload_time_list[3] == '오전':
        ap = 'AM'
    else:
        ap = 'PM'
    last_upload_time = y_m_d + ' ' + ap + ' ' + last_upload_time_list[4]

    # 3. 뉴스 데이터의 기사 작성자
    author = None

    # 4. 뉴스 데이터의 본문
    div = soup.find('div', id='article')
    news_content = div.find_all('p')

    # 뉴스 데이터 본문의 데이터 전처리1
    if re.search('(읽기|provided|[a-z0-9]@[a-z0-9])', news_content[-1].text, flags=re.IGNORECASE):
        del news_content[-1]
    # 뉴스 데이터 본문의 데이터 전처리2
    text = ''.join(t.text for t in news_content).strip(r' \t\n\r\f\v')
    text = re.sub(r'([가-힣])\.(\w)', r'\g<1>. \g<2>', text)
    text = re.sub('(\xa0)+', ' ', text)

    info['news_title'] = title
    info['news_first_upload_time'] = first_upload_time
    info['news_last_upload_time'] = last_upload_time
    info['author'] = author
    info['news_content'] = text
    info['news_url'] = url
    info['news_website'] = 'Investing'

    return info

# Main

In [ ]:
web_page = 'https://www.hankyung.com/koreamarket/news/crypto'
results = []

html = requests.get(web_page).text
soup = BeautifulSoup(html, 'html.parser')
url_tag_list = soup.find_all('h2', {"class": "news-tit"})
info = {} # 뉴스 데이터 정보 Dictionary

# news_url를 찾아서 requests로 HTML GET을 한 다음, BeautifulSoup로 parser
for news_url in url_tag_list:
    url = news_url.find("a")["href"]
    html = requests.get(url).text
    time.sleep(0.1)
    soup = BeautifulSoup(html, 'html.parser')

    # 1. 뉴스 데이터의 제목
    title = soup.find('h1', class_='headline').text.strip(' \t\n\r\f\v')

    # 2. 뉴스 데이터의 최초 업로드 시각과 최종 수정 시각
    first_upload_time = None
    span = soup.find_all('span', class_='txt-date')
    first_upload_time = span[0].text
    last_upload_time = span[1].text

    first_upload_time = datetime.strptime(first_upload_time, '%Y.%m.%d %H:%M')
    first_upload_time = datetime.strftime(first_upload_time, '%Y-%m-%d %p %H:%M')
    last_upload_time = datetime.strptime(last_upload_time, '%Y.%m.%d %H:%M')
    last_upload_time = datetime.strftime(last_upload_time, '%Y-%m-%d %p %H:%M')

    # # 3. 뉴스 데이터의 기사 작성자
    author = None

    # # 4. 뉴스 데이터의 본문
    # div = soup.find('div', id='article')
    # news_content = div.find_all('p')

    # # 뉴스 데이터 본문의 데이터 전처리1
    # if re.search('(읽기|provided|[a-z0-9]@[a-z0-9])', news_content[-1].text, flags=re.IGNORECASE):
    #     del news_content[-1]
    # # 뉴스 데이터 본문의 데이터 전처리2
    # text = ''.join(t.text for t in news_content).strip(r' \t\n\r\f\v')
    # text = re.sub(r'([가-힣])\.(\w)', r'\g<1>. \g<2>', text)
    # text = re.sub('(\xa0)+', ' ', text)

    # info['news_title'] = title
    # info['news_first_upload_time'] = first_upload_time
    # info['news_last_upload_time'] = last_upload_time
    # info['author'] = author
    # info['news_content'] = text
    # info['news_url'] = url
    # info['news_website'] = 'Hankyung'

    break

2024-12-05 AM 07:51
2024-12-05 AM 07:51


In [ ]:
web_page = 'https://www.hankyung.com/koreamarket/news/crypto'
results = []

for i in range(2, 2001):
    html = requests.get(web_page).text
    soup = BeautifulSoup(html, 'html.parser')
    url_tag_list = soup.find_all('h2', {"class": "news-tit"})

    try:
        for url_tag in url_tag_list:
            data = hankyung(url_tag)
            results.append(data)
    except Exception as e:
        print(e)
    
    time.sleep(random.uniform(0.25, 1))
    web_page = f'https://www.hankyung.com/koreamarket/news/crypto?page={i}'